In [1]:
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

## Regular expressions

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_British_scientists"
headers = {"User-Agent" : "Mozilla/5.0"}
req = urllib.request.Request(url, headers=headers)

fp = urllib.request.urlopen(req)
text = fp.read()
text = text.decode("utf8")
fp.close()

In [3]:
with open("HTML.html", "w", encoding='utf-8') as file:
    file.write(text)

scientists = re.findall("<li>.*</li>", text)
#<li><a href="/wiki/Alcuin" title="Alcuin">Alcuin</a> (735-804), scholar and theologian</li>

In [4]:
def get_names(scientists):
    names = np.array([])
    for scientist in scientists:
        name = re.findall("\">.*?</a>", scientist)
        real_len = len(name[0]) - 4
        name = name[0][2:real_len]
        names = np.append(names, name)
    return(names)

In [5]:
def get_links(scientists):
    links = np.array([])
    for scientist in scientists:
        link = re.findall("/wiki/.*?\"", scientist)
        real_len = len(link[0]) - 1
        link = 'https://en.wikipedia.org/' + link[0][0:real_len]
        links = np.append(links, link)
    return(links)

In [6]:
def get_universities(url):
    headers ={ "User-Agent" : "Mozilla/5.0" }
    req = urllib.request.Request(url, None, headers)
    fp = urllib.request.urlopen(req)
    text = fp.read()
    text = text.decode("utf8")
    fp.close()
    
    infobox = re.findall(r"infobox biography vcard((?:(?!</table>).|\n)*)</table>", text)
    if len(infobox) != 0:
        institutions = re.findall(r"Institutions((?:(?!</td>).|\n)*)</td>", infobox[0])
        if len(institutions) != 0:
            return re.findall(r"title=\"((?:(?!\").)*)\"", institutions[0])
    return []

In [7]:
names = get_names(scientists)
with open("ScientistsNamesRe.txt", 'w') as sci_names:
    for name in names: 
        sci_names.write(str(name.encode("utf8")))
        sci_names.write('\n')

In [8]:
links = get_links(scientists)
with open("ScientistsLinksRe.txt", 'w') as sci_links:
    for link in links:
        sci_links.write(link)
        sci_links.write('\n')

## Getting connections

In [9]:
universities = set([])
universities_connections = []
for i in range(len(scientists)):
    print(i)
    cur_universities = get_universities(links[i])
    for university in cur_universities:
        universities.add(university)
    
    for university_a in cur_universities:
        for university_b in cur_universities:
            if university_a != university_b:
                universities_connections.append((university_a, university_b, names[i]))
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
universities = list(universities)
with open("InstituteRelationsRe.tgf", "w", encoding='utf-8') as file:
    for i in range(len(universities)):
        file.write(str(i+1) + " " + str(universities[i]) + "\n")
    
    file.write("#\n")
    
    for connection in universities_connections:
        file.write(str(universities.index(connection[0])) + " " + str(universities.index(connection[1])) + " " + connection[2] + "\n")  

## Beatiful Soup

In [11]:
url = "https://en.wikipedia.org/wiki/List_of_British_scientists"
headers ={ "User-Agent" : "Mozilla/5.0" }
req = urllib.request.Request(url, headers=headers)
fp = urllib.request.urlopen(req)

text = fp.read()
text = text.decode("utf8")

fp.close()

In [12]:
soup = BeautifulSoup(text, 'html.parser')
links = []
names = []
for link in soup.find_all('a', {'href': re.compile('/wiki/[A-Z]+')}):
    links.append(link.get('href'))
links = links[links.index('/wiki/Alcuin'):links.index('/wiki/Help:Category')]
links = ['https://en.wikipedia.org/' + link for link in links]

names = [elm.get_text() for elm in soup.find_all('a', {'href': re.compile('/wiki/[A-Z]+')})]
names = names[names.index('Alcuin'):names.index('Thomas Young')+1]

links = [links[i] for i in range(len(names)) if names[i][0].isupper()]
names = [names[i] for i in range(len(names)) if names[i][0].isupper()]

In [13]:
with open("ScientistsNamesB4s.txt", 'w') as sci_names:
    for name in names: 
        sci_names.write(str(name.encode("utf8")))
        sci_names.write('\n')

In [14]:
with open("ScientistsLinksB4s.txt", 'w') as sci_links:
    for link in links:
        sci_links.write(link)
        sci_links.write('\n')

## Getting connections

In [15]:
def get_universities(url):
    institutions = []
    headers ={ "User-Agent" : "Mozilla/5.0" }
    req = urllib.request.Request(url, None, headers)
    fp = urllib.request.urlopen(req)
    text = fp.read()
    text = text.decode("utf8")
    fp.close()
    
    soup = BeautifulSoup(text, 'html.parser')
    infobox = soup.find(class_='infobox biography vcard')
    res = []
    if infobox:
        for tr in infobox.find_all('tr'):
            if tr.find(string='Institutions'):
                for university in tr.td.find_all('a'):
                    name = university.get_text()
                    if name[0].isupper():
                        res.append(name)
                        
    return res

In [16]:
universities = set([])
universities_connections = []
for i in range(len(scientists)):
    print(i)
    cur_universities = get_universities(links[i])
    for university in cur_universities:
        universities.add(university)
    
    for university_a in cur_universities:
        for university_b in cur_universities:
            if university_a != university_b:
                universities_connections.append((university_a, university_b, names[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

IndexError: string index out of range

In [115]:
universities = list(universities)
with open("InstituteRelationsB4s.tgf", "w", encoding='utf-8') as file:
    for i in range(len(universities)):
        file.write(str(i+1) + " " + str(universities[i]) + "\n")
    
    file.write("#\n")
    
    for connection in universities_connections:
        file.write(str(universities.index(connection[0])) + " " + str(universities.index(connection[1])) + " " + connection[2] + "\n")  